## London House Price Predictive Model
### Imports

In [3]:
import pandas as pd
from datetime import datetime as dt, timezone as tz
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv('data/kaggle_london_house_price_data.csv')

In [3]:
df.dtypes

fullAddress                                   object
postcode                                      object
country                                       object
outcode                                       object
latitude                                     float64
longitude                                    float64
bathrooms                                    float64
bedrooms                                     float64
floorAreaSqM                                 float64
livingRooms                                  float64
tenure                                        object
propertyType                                  object
currentEnergyRating                           object
rentEstimate_lowerPrice                      float64
rentEstimate_currentPrice                    float64
rentEstimate_upperPrice                      float64
saleEstimate_lowerPrice                      float64
saleEstimate_currentPrice                    float64
saleEstimate_upperPrice                      f

### Data Cleaning

In [4]:
df['sale_date'] = pd.to_datetime(df['saleEstimate_ingestedAt'])

In [5]:
key_df = df.drop(columns=['history_numericChange', 'history_percentageChange', 'history_date', 'history_price','saleEstimate_valueChange.saleDate','saleEstimate_valueChange.percentageChange','saleEstimate_ingestedAt','saleEstimate_valueChange.numericChange'])

In [6]:
house_df = key_df[key_df['sale_date'] > dt(2015, 1,1).replace(tzinfo=tz.utc)] #Only considering houses with estimated values in the last 10 years

### EDA

In [7]:
house_df.isna().sum()

fullAddress                         0
postcode                            0
country                             0
outcode                             0
latitude                            0
longitude                           0
bathrooms                       77755
bedrooms                        40404
floorAreaSqM                    25066
livingRooms                     60341
tenure                          11494
propertyType                     1126
currentEnergyRating             84288
rentEstimate_lowerPrice          1101
rentEstimate_currentPrice        1101
rentEstimate_upperPrice          1101
saleEstimate_lowerPrice             0
saleEstimate_currentPrice           0
saleEstimate_upperPrice             0
saleEstimate_confidenceLevel        0
sale_date                           0
dtype: int64

In [8]:
temp_df = house_df.dropna(subset=['bathrooms', 'livingRooms','bedrooms','floorAreaSqM'])
temp_df.isna().sum() # Cannot impute the number of rooms and area of a building

fullAddress                         0
postcode                            0
country                             0
outcode                             0
latitude                            0
longitude                           0
bathrooms                           0
bedrooms                            0
floorAreaSqM                        0
livingRooms                         0
tenure                           1702
propertyType                        0
currentEnergyRating             51266
rentEstimate_lowerPrice            16
rentEstimate_currentPrice          16
rentEstimate_upperPrice            16
saleEstimate_lowerPrice             0
saleEstimate_currentPrice           0
saleEstimate_upperPrice             0
saleEstimate_confidenceLevel        0
sale_date                           0
dtype: int64

In [9]:
flat_df = temp_df[temp_df['propertyType'].str.lower().str.contains('flat')]
flat_df['tenure'].value_counts() # 98% of flats are leasehold. Would be therefore valid to impute that any flats w/o tenure are in fact leasehold

tenure
Leasehold    178804
Feudal         2796
Freehold       2039
Shared          451
Name: count, dtype: int64

In [10]:
flat_df['currentEnergyRating'].value_counts() #Normally distributed around C/D could impute with preference for B,C,D,E for flats

currentEnergyRating
C    69632
D    49279
B    22602
E     9409
F      586
A      105
G      101
Name: count, dtype: int64

In [11]:
non_flat_df = temp_df[~temp_df['propertyType'].str.lower().str.contains('flat')]

In [12]:
non_flat_df['tenure'].value_counts() #Houses tend to be freehold with 97% of houses Freehold, impute freehold for non flat properties

tenure
Freehold     128580
Leasehold      4128
Feudal           56
Shared           22
Name: count, dtype: int64

In [13]:
non_flat_df['currentEnergyRating'].value_counts() #Sharper normal distribution centered around D, could impute with preference for C,D,E

currentEnergyRating
D    64439
C    32420
E    15526
B     1714
F     1061
G      329
A      109
Name: count, dtype: int64

This exploration of the data has clear reasoning for imputing some fields and removing rows with missing key fields.
- Bathrooms : Cannot be reliably imputed and is necessary for the future model
- Living Rooms: Cannot be reliably imputed and is necessary for the future model
- Floor Area : Cannot be reliably imputed and is necessary for the future model
- Tenure: Can be reliably imputed and is necessary for the future model
- Energy rating: Can be reliably imputed and is necessary for the future model

### Pre-Processing

#### Imputation

In [15]:
sales_df = house_df.dropna(subset=['bathrooms', 'livingRooms', 'bedrooms', 'floorAreaSqM']) 
sales_df.drop(columns=['rentEstimate_lowerPrice', 'rentEstimate_currentPrice', 'rentEstimate_upperPrice', 'sale_date','saleEstimate_confidenceLevel'],inplace=True)
sales_df['tenure'] = sales_df['propertyType'].apply(lambda x: 'Leasehold' if 'flat' in x.lower() else 'Freehold')

C:\Users\Alex.Barnes\AppData\Local\Temp\ipykernel_29524\2432086349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df.drop(columns=['rentEstimate_lowerPrice', 'rentEstimate_currentPrice', 'rentEstimate_upperPrice', 'sale_date','saleEstimate_confidenceLevel'],inplace=True)
C:\Users\Alex.Barnes\AppData\Local\Temp\ipykernel_29524\2432086349.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['tenure'] = sales_df['propertyType'].apply(lambda x: 'Leasehold' if 'flat' in x.lower() else 'Freehold')


In [16]:
sales_df.columns

Index(['fullAddress', 'postcode', 'country', 'outcode', 'latitude',
       'longitude', 'bathrooms', 'bedrooms', 'floorAreaSqM', 'livingRooms',
       'tenure', 'propertyType', 'currentEnergyRating',
       'saleEstimate_lowerPrice', 'saleEstimate_currentPrice',
       'saleEstimate_upperPrice'],
      dtype='object')

In [17]:
flats = sales_df[sales_df['propertyType'].str.lower().str.contains('flat')]
houses = sales_df[~sales_df['propertyType'].str.lower().str.contains('flat')]

In [18]:
for group in [flats, houses]:
    mean_rating = group['currentEnergyRating'].dropna().astype(str).mode()[0]
    std_dev = 1
    missing_idx = group['currentEnergyRating'].isna()
    group.loc[missing_idx, 'currentEnergyRating'] = np.random.choice(
        [mean_rating], size=missing_idx.sum())

In [19]:
df = pd.concat([flats, houses])

In [6]:
numerical_features = ['bathrooms', 'bedrooms', 'floorAreaSqM', 'livingRooms']
categorical_features = ['tenure', 'propertyType', 'currentEnergyRating']

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
])

models = {
    "Ridge": Ridge(),
    "SVR": SVR(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor()
}

param_grids = {
        "Ridge": {
        'model__alpha': [0.1, 1.0, 10.0]
    },
    "SVR": {
        'model__C': [1, 10],
        'model__kernel': ['linear', 'rbf']
    },
    "RandomForest": {
        'model__n_estimators': [50, 100],
        'model__max_depth': [10, 20],
        'model__max_features': ['sqrt', 'log2'],
        'model__min_samples_split': [5, 10],
    },
    "GradientBoosting": {
        'model__n_estimators': [100, 300],
        'model__learning_rate': [0.05, 0.1]
    }
}

In [23]:
df.to_pickle("data/house_df.pkl")

In [4]:
df = pd.read_pickle("data/house_df.pkl")

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    df[categorical_features+numerical_features], df['saleEstimate_currentPrice'], test_size=0.2, random_state=42)

In [126]:
def train_model(name,model,param_grids,preprocessor,best_models):
    print(f"Training {name}...")
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    grid_search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_grids[name],
        n_iter=10,
        scoring="neg_mean_absolute_error",
        cv=5,
        n_jobs=10
    )
    grid_search.fit(X_train.iloc[:10000], y_train.iloc[:10000])
    best_models[name] = grid_search.best_estimator_

    print(f"Best parameters for {name}: {grid_search.best_params_}")
    return best_models

In [ ]:
best_models = {}
for name,model in models.items():
    best_models = train_model(name,model,param_grids,preprocessor,best_models)

In [ ]:
CACHE_DIR = "model_cache"
os.makedirs(CACHE_DIR, exist_ok=True)
def train_model_presets(name, model, preprocessor, best_models):
    cache_path = os.path.join(CACHE_DIR, f"{name}.pkl")
    if os.path.exists(cache_path):
        print(f"Loading cached model for {name}...")
        with open(cache_path, "rb") as f:
            pipeline = pickle.load(f)
    else:
        print(f"Training {name}...")
        pipeline = Pipeline([
            ("preprocessor", preprocessor),
            ("model", model)
        ])
        pipeline.fit(X_train, y_train)
        
        # Save the trained model
        with open(cache_path, "wb") as f:
            pickle.dump(pipeline, f)
        
        print(f"Training complete for {name}. Model cached.")
    return name, pipeline

In [ ]:
models = [
    ("GradientBoosting", GradientBoostingRegressor(
        n_estimators=300, learning_rate=0.05)),
    ("RandomForest", RandomForestRegressor(n_estimators=50, min_samples_split=5,
                                           max_features='sqrt', max_depth=10, n_jobs=-1)),
    ("Ridge", Ridge(alpha=10)),
    ("SVR", SVR(kernel='linear', C=10))
]

best_models = {}
for name, model in models:
    best_models = train_model_presets(name, model, preprocessor, best_models)

Loading cached model for GradientBoosting...
GradientBoosting
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['bathrooms', 'bedrooms',
                                                   'floorAreaSqM',
                                                   'livingRooms']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['tenure', 'propertyType',
                                                   'currentEnergyRating'])])),
                ('model',
                 GradientBoostingRegressor(learning_rate=0.05,
                                           n_estimators=300))])
Loading cached model for RandomForest...
RandomForest
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
 

In [ ]:
def evaluate_model(model_path, X_test, y_test):
    with open(f'model_cache/{model_path}', 'rb') as f:
        model = pickle.load(f)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"🔹 {model} Performance:")
    print(f"   MAE: {mae:.2f}")
    print(f"   RMSE: {rmse:.2f}")
    print(f"   R² Score: {r2:.2f}\n")

    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=y_test,
                    y=y_pred, alpha=0.5)
    plt.plot([y_test.min().min(), y_test.max().max()], [y_test.min().min(
    ), y_test.max().max()], color='red', linestyle='--')
    plt.xlabel("True Sale Estimate Price")
    plt.ylabel("Predicted Sale Estimate Price")
    plt.title(f"{model} - True vs. Predicted Prices")
    plt.show()

In [ ]:
for path in os.listdir('model_cache'):
    evaluate_model(path, X_test,y_test)

### Feature Investigation
#### Outcode vs Postcode